---
title: "SQL Basics using DuckDB"
date: "2023-08-20"
categories: [tutorial]
format:
  html:
    code-fold: false
    df-print: paged
image: duckdb_logo.jpeg
---

I've been wanting to put together a simple resource for learning the basics of querying data with SQL for some time now. However, one of the roadblocks has been the overhead of initial setup instructions for creating a database to query from. Even cloud options have hurdles that prevent just jumping in and writing SQL.

## DuckDB: Simple and Effective
One of my favorite new tools is [DuckDB](https://duckdb.org/). Especially when combined with pandas dataframes. It enables me to use pandas to quickly read a data source into a dataframe and then shift away from the often confusing array of pandas querying syntax and shift back to something I am most comfortable with in my data analysis toolbelt: SQL.

To start, we'll need some data to work with. Similar to the [last tutorial](/posts/pandas-yaml-config/), let's download the [2021-2022 NCES national school directory](https://nces.ed.gov/ccd/Data/zip/ccd_sch_029_2122_w_1a_071722.zip) to work with.

In [27]:
import pandas as pd


# directory = pd.read_csv("../pandas-yaml-config/ccd_sch_029_2122_w_1a_071722.csv", dtype=str)
directory = pd.read_csv("ccd_sch_029_2122_w_1a_071722.csv", dtype=str)


# convert column names to lowercase, 
# because I'm not a fan of my column names screaming at me in ALL CAPS
directory.columns = [c.lower() for c in df.columns] 

directory.head()

,school_year,fipst,statename,st,sch_name,lea_name,state_agency_no,union,st_leaid,leaid,...,g_10_offered,g_11_offered,g_12_offered,g_13_offered,g_ug_offered,g_ae_offered,gslo,gshi,level,igoffered
0,2021-2022,01,ALABAMA,AL,Albertville Middle School,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,07,08,Middle,As reported
1,2021-2022,01,ALABAMA,AL,Albertville High School,Albertville City,01,NaN,AL-101,0100005,...,Yes,Yes,Yes,No,No,No,09,12,High,As reported
2,2021-2022,01,ALABAMA,AL,Albertville Intermediate School,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,05,06,Middle,As reported
3,2021-2022,01,ALABAMA,AL,Albertville Elementary School,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,03,04,Elementary,As reported
4,2021-2022,01,ALABAMA,AL,Albertville Kindergarten and PreK,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,PK,KG,Elementary,As reported


## A quick pause for vocabulary

Before we dive in to writing SQL queries, let's pause to define a few common terms that will come up.

- **Table**: A table is a fundamental component of a database. It represents a collection of related data organized in rows and columns. Each row in a table is called a record, and each column represents a field.
- **Field**: A field is a single piece of data within a table. It corresponds to the columns in a table and defines the type of data it can hold, such as text, numbers, dates, or binary data.
- **Record**: A record, also known as a row, is a complete set of related data in a table. It contains values for each field, representing a single entity or data entry within the database.
- **Schema**: A schema is a blueprint or structure that defines the organization of a database. It outlines the tables, fields, data types, relationships, and constraints that form the database's structure.
- **Query**: A query is a request made to the database to retrieve or manipulate data. It uses a structured query language (SQL) to interact with the database.

### Understanding table schema
Understanding the schema of a table is essential before writing a SQL query. The schema defines the structure of the table, including the names of columns, data types, and constraints. When exploring unfamiliar datasets, understanding the schema provides insights into the available data and helps you identify relevant tables and columns for analysis. It also ensures that you refer to the correct column names and use the appropriate data types in your SQL query, leading to accurate and valid results. Knowing the data types of each column helps you format your query correctly. 

For example, when dealing with dates or numerical values, understanding the data types ensures you use the right functions and operators for calculations. When writing queries, you often need to filter or sort data based on certain criteria. Understanding the schema allows you to apply the correct conditions and sorting instructions to retrieve the desired results effectively.

Now, let's orient ourselves to the data in this dataset and it's schema. To do so, let's get a list of columns and get a sense of the data volume.

In [28]:
directory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102130 entries, 0 to 102129
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   school_year          102130 non-null  object
 1   fipst                102130 non-null  object
 2   statename            102130 non-null  object
 3   st                   102130 non-null  object
 4   sch_name             102130 non-null  object
 5   lea_name             102130 non-null  object
 6   state_agency_no      102130 non-null  object
 7   union                2440 non-null    object
 8   st_leaid             102130 non-null  object
 9   leaid                102130 non-null  object
 10  st_schid             102130 non-null  object
 11  ncessch              102130 non-null  object
 12  schid                102130 non-null  object
 13  mstreet1             102130 non-null  object
 14  mstreet2             1672 non-null    object
 15  mstreet3             2 non-null   

This shows us a number of things:
- all the column names
- their data types (recall we have set all of them to strings, which pandas lists as `object`)
- the number of non-null values in each column
- the number of columns: 65
- the number of rows: 102,130
- it even shows the amount of memory used to hold this data: 50.6 MB

### A note about null
In SQL and databases, `null` represents the absence of a value or unknown data for a specific field in a table. It's not the same as zero or an empty string and requires special handling in SQL queries. The `null` value can be used in any data type, whether it's a string, numeric, date, or any other data type.

Dealing with null values in SQL queries can be tricky, and several common mistakes can lead to unexpected or incorrect results. 


## Getting Started
Now that we have a basic lay-of-the-land for our data, we can begin querying it. At this point, I will shift away from pandas syntax and use SQL queries.

To start, let's install duckdb.

```
pip install duckdb
```

In [29]:
import duckdb

query = "select * from directory limit 5"

duckdb.sql(query).df()

,school_year,fipst,statename,st,sch_name,lea_name,state_agency_no,union,st_leaid,leaid,...,g_10_offered,g_11_offered,g_12_offered,g_13_offered,g_ug_offered,g_ae_offered,gslo,gshi,level,igoffered
0,2021-2022,01,ALABAMA,AL,Albertville Middle School,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,07,08,Middle,As reported
1,2021-2022,01,ALABAMA,AL,Albertville High School,Albertville City,01,NaN,AL-101,0100005,...,Yes,Yes,Yes,No,No,No,09,12,High,As reported
2,2021-2022,01,ALABAMA,AL,Albertville Intermediate School,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,05,06,Middle,As reported
3,2021-2022,01,ALABAMA,AL,Albertville Elementary School,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,03,04,Elementary,As reported
4,2021-2022,01,ALABAMA,AL,Albertville Kindergarten and PreK,Albertville City,01,NaN,AL-101,0100005,...,No,No,No,No,No,No,PK,KG,Elementary,As reported


This basic query is equivalent to `directory.head()`.

```sql
select * 
from directory
limit 5
```

What is each keyword doing here?

`select`:
The select keyword is used to specify which columns or expressions to include in the query result. It allows you to choose the data you want to retrieve from the tables in the database. In SQL, the asterisk (`*`) is used in a `select` statement to represent all columns in a table. 

`from`:
The from keyword is used to specify the table from which the data will be retrieved. It tells the query where to find the data to be selected. With duckdb, we can specify a dataframe as the table to be queried.

`limit`:
The limit keyword is used to restrict the number of rows returned in the query result. It is particularly useful when you only need to see a small subset of the data.

## Asking Questions
SQL Queries are fundamentally a way of interrogating your data. You usually start with a question you want to answer and then form your query (question) is such a way that the database can understand it and return the relevant data. 

One of the basic ways to interrogate the data is to constrain it in some way. Filtering the data to show only a subset allows you to begin to identify possible patterns. The `where` clause is particularly valuable for this purpose. 

Let's translate this question into a SQL query: _Which schools in Delaware offer 12th grade enrollment?_

In [30]:
query = """
  select sch_name
  from directory
  where statename = 'DELAWARE'
    and g_12_offered = 'Yes'
  order by sch_name
"""

duckdb.sql(query).df()

,sch_name
0,Appoquinimink High School
1,Brandywine High School
2,Brandywine SITE
3,Brennen School (The)
4,Caesar Rodney High School
5,Calloway (Cab) School of the Arts
6,Cape Henlopen High School
7,Charter School of Wilmington
8,Christiana High School
9,Concord High School


_NOTE:_ You may have noticed the use of `"""` in the code above. This enables a string to be broken out across multiple lines which I find easier to read for SQL queries.

What is happening here? Let's break down this query.

```sql
select sch_name -- return only the school name column
from directory -- from the data in our dataframe
where statename = 'DELAWARE' -- where the state name is DELAWARE (state names are all caps in the dataset)
and g_12_offered = 'Yes' -- and where grade 12 is offered
order by sch_name -- sort the resulting data alphabetically by the school name
```


Next, let's refine this query to identify which of those schools are charter schools or not.

In [31]:
query = """
  select 
    sch_name,
    charter_text,
  from directory
  where statename = 'DELAWARE'
    and g_12_offered = 'Yes'
  order by sch_name
"""

duckdb.sql(query).df()

,sch_name,charter_text
0,Appoquinimink High School,No
1,Brandywine High School,No
2,Brandywine SITE,No
3,Brennen School (The),No
4,Caesar Rodney High School,No
5,Calloway (Cab) School of the Arts,No
6,Cape Henlopen High School,No
7,Charter School of Wilmington,Yes
8,Christiana High School,No
9,Concord High School,No


And if we want to limit our list to just the charter schools, we can add another filter to the `where` clause.

In [32]:
query = """
  select 
    sch_name,
    charter_text,
  from directory
  where statename = 'DELAWARE'
    and g_12_offered = 'Yes'
    and charter_text = 'Yes'
  order by sch_name
"""

duckdb.sql(query).df()

,sch_name,charter_text
0,Charter School of Wilmington,Yes
1,Delaware Military Academy,Yes
2,Early College High School at Del State,Yes
3,First State Military Academy,Yes
4,Freire Charter School,Yes
5,Great Oaks Charter School,Yes
6,MOT Charter School,Yes
7,Newark Charter School,Yes
8,Odyssey Charter School,Yes
9,Positive Outcomes Charter School,Yes


## Counting: A common type of question to ask about data

After this initial investigation, my curiosity has been piqued. I am wondering how many schools are charters vs. traditional public schools in Delaware (regardless of grades offered). Let's translate that question into a sql query using some basic aggregation.

In [33]:
query = """
  select 
    statename,
    charter_text,
    count(sch_name) as num_schools,
  from directory
  where statename = 'DELAWARE'
  group by 
    statename, 
    charter_text
"""

duckdb.sql(query).df()

,statename,charter_text,num_schools
0,DELAWARE,Yes,23
1,DELAWARE,No,208


How does this work? 

The `count()` function, predictably, will count the number of values in a given column (or all records if the `*` is passed instead). You can further expand this to count only unique school names by using `count(distinct sch_name)`.  

We are also aliasing the count using the `as` keyword to give the resulting column a name: `num_schools`.

The `group by` clause is necessary for this aggregation because it defines the context in which the `count()` function operates. Without the `group by` clause, the `count()` function would treat the entire result set as a single group. 

Consider this example instead (without grouping):

In [34]:
query = """
  select 
    count(sch_name) as num_schools,
  from directory
  where statename = 'DELAWARE'
"""

duckdb.sql(query).df()

,num_schools
0,231


In order to get a count for each value of the _Yes/No_ flag in the `charter_text` column, we need to group our counts by each of those results. In addition, in order to include the `statename` in our result set, we need to include it in the grouping as well. 

If we wanted to determine the same counts for all states in the dataset, we could simply remove the `where` clause limiting the data to Delaware.

In [35]:
query = """
  select 
    statename,
    charter_text,
    count(sch_name) as num_schools,
  from directory
  group by 
    statename, 
    charter_text
"""

duckdb.sql(query).df()

,statename,charter_text,num_schools
0,ALABAMA,No,1553
1,ALABAMA,Yes,13
2,ALASKA,No,478
3,ALASKA,Yes,31
4,ARIZONA,No,1909
...,...,...,...
100,GUAM,Yes,3
101,NORTHERN MARIANAS,Not applicable,35
102,PUERTO RICO,No,846
103,PUERTO RICO,Yes,7


This gives us some sense of the breakdown, but it actually might be useful to rephrase our question as: _What number and percent of schools in each state are charter schools?_

With that additional clarity, we can refine our query a bit using some conditional logic.

In [36]:
query = """
  select 
    statename,
    count(
        if(charter_text = 'Yes', sch_name, null)
    ) as num_charter_schools,
    count(sch_name) as num_schools,
  from directory
  group by statename
  order by statename
"""

duckdb.sql(query).df()

,statename,num_charter_schools,num_schools
0,ALABAMA,13,1566
1,ALASKA,31,509
2,AMERICAN SAMOA,0,29
3,ARIZONA,606,2515
4,ARKANSAS,93,1099
5,BUREAU OF INDIAN EDUCATION,0,174
6,CALIFORNIA,1337,10456
7,COLORADO,268,1960
8,CONNECTICUT,21,1009
9,DELAWARE,23,231


That gets us the counts of charter schools in each state as well as the total number of schools in each state. Let's breakdown what this is doing:

```sql
count(
  if(charter_text = 'Yes', sch_name, null)
) as num_charter_schools,
```

The `if()` function checks whether a statement evaluates to `true` or not and then returns one value for `true` and another for `false`. Here, we are leveraging that functionality to count the school name if the value of the `charter_text` column is `Yes` but returning `null` if not. That works because the `count()` function ignores `null` values when aggregating. 

A similar, but slightly different approach could be done using the `sum()` function instead:

```sql
sum(
  if(charter_text = 'Yes', 1, 0)
) as num_charter_schools,
```

This would treat each charter school as being equivalent to the value `1` and each non-charter schools as equivalent to the value `0` and then would add them up. The result is the same in this case. There can be subtle differences in the approach depending on the data being aggregated. It can be useful, at times, to compare the resulting differences (if any).

### Calculating the percentage of schools
We have our counts now, but we don't have the percentage of schools and that is ultimately the question we are asking. To determine that, we need to divide the number of charters in each state by the total number of schools.

We could do that like this:

In [37]:
query = """
  select 
    statename,
    count(if(charter_text = 'Yes', sch_name, null)) as num_charter_schools,
    count(sch_name) as num_schools,
    count(if(charter_text = 'Yes', sch_name, null))/count(sch_name) as percent_of_schools,
  from directory
  group by statename
  order by statename
"""

duckdb.sql(query).df()

,statename,num_charter_schools,num_schools,percent_of_schools
0,ALABAMA,13,1566,0.008301
1,ALASKA,31,509,0.060904
2,AMERICAN SAMOA,0,29,0.000000
3,ARIZONA,606,2515,0.240954
4,ARKANSAS,93,1099,0.084622
5,BUREAU OF INDIAN EDUCATION,0,174,0.000000
6,CALIFORNIA,1337,10456,0.127869
7,COLORADO,268,1960,0.136735
8,CONNECTICUT,21,1009,0.020813
9,DELAWARE,23,231,0.099567


Not bad, but I find that decimal representation of the percentage a bit harder to read. Let's transform that to look like `12.5%` by:
- multiplying the decimal by 100 to get a whole number
- rounding to a single decimal digit
- converting it to a string and adding the `%` sign

In [38]:
query = """
  select 
    statename,
    count(if(charter_text = 'Yes', sch_name, null)) as num_charter_schools,
    count(sch_name) as num_schools,
    concat(cast(round((count(if(charter_text = 'Yes', sch_name, null))/count(sch_name)) * 100, 1) as string), '%') as percent_of_schools,
  from directory
  group by statename
  order by statename
"""

duckdb.sql(query).df()

,statename,num_charter_schools,num_schools,percent_of_schools
0,ALABAMA,13,1566,0.8%
1,ALASKA,31,509,6.1%
2,AMERICAN SAMOA,0,29,0.0%
3,ARIZONA,606,2515,24.1%
4,ARKANSAS,93,1099,8.5%
5,BUREAU OF INDIAN EDUCATION,0,174,0.0%
6,CALIFORNIA,1337,10456,12.8%
7,COLORADO,268,1960,13.7%
8,CONNECTICUT,21,1009,2.1%
9,DELAWARE,23,231,10.0%


## Cleaning this up for readability

At this point we have a working query, but the syntax is getting a bit dense and hard to read. I'd like to break this up a bit to make it more clear what is happening. 

Our code has three main steps:
1. calculate the counts of charter schools and all schools for each state
2. divide the counts to calculate a percentage value as decimal
3. transform the decimal value to a string representation of the percentage

We can use nested subqueries to do this, but I consider that an anti-pattern in SQL and prefer the use of CTEs (common table expressions). They allow us to designate a query to be an intermediate result that can be further queried.

### New Vocabulary
- **Common Table Expression (CTE)** is a temporary result set that is defined within the scope of a single SQL statement. It allows you to create a named query that can be referenced multiple times within the main query. CTEs are primarily used to improve the readability, modularity, and reusability of SQL queries

### _Note: Subqueries as an anti-pattern_
Using nested subqueries in SQL can become an anti-pattern because it can lead to complex, hard-to-maintain queries and potential performance issues. Nested subqueries involve placing one query inside another, often making the overall query difficult to read, understand, and optimize. While nested subqueries can solve specific problems, using them excessively can lead to unreadable, inefficient, and hard-to-maintain SQL queries. It's generally better to explore alternative techniques like joins, CTEs, and window functions for clearer, more optimized code.



In [39]:
query = """
  with school_counts as (
    select 
      statename,
      count(if(charter_text = 'Yes', sch_name, null)) as num_charter_schools,
      count(sch_name) as num_schools,
    from directory
    group by statename
  ),

  percents as (
      select
        statename,
        num_charter_schools,
        num_schools,
        round((num_charter_schools/num_schools) * 100, 1) as percent_of_schools
      from school_counts
  )

  select
    statename,
    num_charter_schools,
    num_schools,
    concat(cast(percent_of_schools as string),'%') as percent_of_schools
  from percents
"""

charters = duckdb.sql(query).df()

Another way to do this that is unique to duckdb would be to store each query into a variable and execute queries against those. 

In [40]:
query = """
  select 
      statename,
      count(if(charter_text = 'Yes', sch_name, null)) as num_charter_schools,
      count(sch_name) as num_schools,
  from directory
  group by statename
"""
school_counts = duckdb.sql(query).df()

query = """
  select
    statename,
    num_charter_schools,
    num_schools,
    round((num_charter_schools/num_schools) * 100, 1) as percent_of_schools
  from school_counts
"""
percents = duckdb.sql(query).df()

query = """
  select
    statename,
    num_charter_schools,
    num_schools,
    concat(cast(percent_of_schools as string),'%') as percent_of_schools
  from percents
"""
charters = duckdb.sql(query).df()

In [100]:
charters

,statename,num_charter_schools,num_schools,percent_of_schools
0,ALABAMA,13,1566,0.8%
1,ALASKA,31,509,6.1%
2,ARIZONA,606,2515,24.1%
3,ARKANSAS,93,1099,8.5%
4,CALIFORNIA,1337,10456,12.8%
5,COLORADO,268,1960,13.7%
6,CONNECTICUT,21,1009,2.1%
7,DELAWARE,23,231,10.0%
8,DISTRICT OF COLUMBIA,125,243,51.4%
9,FLORIDA,767,4306,17.8%


Either way, we can now see the answer to our initial question. However, at this point a new question has come to mind: _Which states have the highest percentage of charter schools?_

To answer that, we can simply sort the results:

In [102]:
query = """
  select * 
  from charters 
  order by percent_of_schools desc
"""
duckdb.sql(query).df()

,statename,num_charter_schools,num_schools,percent_of_schools
0,OHIO,325,3689,8.8%
1,ARKANSAS,93,1099,8.5%
2,NORTH CAROLINA,223,2765,8.1%
3,NEW HAMPSHIRE,39,498,7.8%
4,SOUTH CAROLINA,93,1288,7.2%
5,NEW YORK,331,4842,6.8%
6,GUAM,3,44,6.8%
7,INDIANA,121,1933,6.3%
8,TENNESSEE,121,1931,6.3%
9,PENNSYLVANIA,184,2963,6.2%


Oh no!!!

Our conversion of the percentage into a string has ruined our ability to sort the values. This is why it is so vital to understand data types. Strings are sorted alphabetically but numbers are sorted numerically. What that means in this context is that when sorting descending (reverse), the string `"8"` is larger than the string `"50"` because 8 comes after 5. If the value were still numbers that would not be the case. 

To resolve this we need to either: preserve the original decimal column for sorting rather than replace it entirely, or do our sorting before the conversion. 

In [42]:
query = """
  with school_counts as (
    select 
      statename,
      count(if(charter_text = 'Yes', sch_name, null)) as num_charter_schools,
      count(sch_name) as num_schools,
    from directory
    group by statename
  ),

  percents as (
      select
        statename,
        num_charter_schools,
        num_schools,
        round((num_charter_schools/num_schools) * 100, 1) as percent_of_schools_numeric
      from school_counts
  )

  select
    statename,
    num_charter_schools,
    num_schools,
    concat(cast(percent_of_schools_numeric as string),'%') as percent_of_schools
  from percents
  order by percent_of_schools_numeric desc
"""

duckdb.sql(query).df()

,statename,num_charter_schools,num_schools,percent_of_schools
0,DISTRICT OF COLUMBIA,125,243,51.4%
1,ARIZONA,606,2515,24.1%
2,FLORIDA,767,4306,17.8%
3,COLORADO,268,1960,13.7%
4,NEVADA,99,769,12.9%
5,RHODE ISLAND,41,319,12.9%
6,CALIFORNIA,1337,10456,12.8%
7,UTAH,141,1123,12.6%
8,HAWAII,37,296,12.5%
9,NEW MEXICO,102,894,11.4%


Now we can see the Washington DC has the highest percent of charter schools by a large margin. That is the legacy of [Michelle Rhee's impact on DC schools after passage of the D.C. Public Education Reform Amendment Act in 2007](https://www.usnews.com/opinion/knowledge-bank/articles/2017-04-20/michelle-rhee-set-national-example-of-education-reform-in-washington-dc).  

## The Devil is in the details

While DC might have the largest _**percent**_ of charter schools it is out of a small number of schools to begin with given the size of Washington DC. If we sort by count we can see that California and Texas have the largest total number of charter schools, but because of the size of the population in those states, the total represents a smaller percentage. 

Washington DC is not even in the top 10.

In [104]:
query = """
  select * 
  from charters 
  order by num_charter_schools desc 
  limit 10
"""
duckdb.sql(query).df()

,statename,num_charter_schools,num_schools,percent_of_schools
0,CALIFORNIA,1337,10456,12.8%
1,TEXAS,1056,9652,10.9%
2,FLORIDA,767,4306,17.8%
3,ARIZONA,606,2515,24.1%
4,MICHIGAN,386,3570,10.8%
5,NEW YORK,331,4842,6.8%
6,OHIO,325,3689,8.8%
7,MINNESOTA,306,2773,11.0%
8,COLORADO,268,1960,13.7%
9,WISCONSIN,243,2263,10.7%


This begs the question: _Which states have the most students in charter schools?_ 

Not all school sizes are equal and there could be a large amount of schools with smaller enrollments or fewer schools with larger enrollments. Unfortunately, our initial dataset doesn't have enrollment information. To determine this, we need new data. Going back to NCES, we can pull the [school membership data](https://nces.ed.gov/ccd/Data/zip/ccd_SCH_052_2122_l_1a_071722.zip) as well. This is a much larger dataset (198 MB compressed) than the directory info but doesn't have as many of the descriptors as our original dataset (such as the charter flag).

To answer this final query, we'll need to combine the data in these two datasets somehow. 

This is where SQL really shines. The sql `join` is one of the fundamental features of the language that makes it so powerful for data analysis. SQL joins are like puzzle pieces that help you combine information from different tables in a database. They're useful when you want to see related data together, such as matching customers with their orders or students with their courses. Instead of keeping all the information in one big table, you can split it into smaller ones and then use joins to bring the pieces together, making it easier to understand and work with your data.

In [8]:
enrollment = pd.read_csv("ccd_SCH_052_2122_l_1a_071722.csv", dtype=str)
enrollment.columns = [c.lower() for c in enrollment.columns]
enrollment.head()

,school_year,fipst,statename,st,sch_name,state_agency_no,union,st_leaid,leaid,st_schid,ncessch,schid,grade,race_ethnicity,sex,student_count,total_indicator,dms_flag
0,2021-2022,01,ALABAMA,AL,Albertville Middle School,01,NaN,AL-101,0100005,AL-101-0010,010000500870,0100870,Grade 7,American Indian or Alaska Native,Female,1,Category Set A - By Race/Ethnicity; Sex; Grade,Reported
1,2021-2022,01,ALABAMA,AL,Albertville Middle School,01,NaN,AL-101,0100005,AL-101-0010,010000500870,0100870,Grade 7,American Indian or Alaska Native,Male,3,Category Set A - By Race/Ethnicity; Sex; Grade,Reported
2,2021-2022,01,ALABAMA,AL,Albertville Middle School,01,NaN,AL-101,0100005,AL-101-0010,010000500870,0100870,Grade 7,Asian,Female,2,Category Set A - By Race/Ethnicity; Sex; Grade,Reported
3,2021-2022,01,ALABAMA,AL,Albertville Middle School,01,NaN,AL-101,0100005,AL-101-0010,010000500870,0100870,Grade 7,Asian,Male,2,Category Set A - By Race/Ethnicity; Sex; Grade,Reported
4,2021-2022,01,ALABAMA,AL,Albertville Middle School,01,NaN,AL-101,0100005,AL-101-0010,010000500870,0100870,Grade 7,Black or African American,Female,7,Category Set A - By Race/Ethnicity; Sex; Grade,Reported


## Exploring Enrollment
My goal here is to simply pull out the school-wide total enrollment for each school, but a quick scan of that dataframe preview indicates this data is pretty granular. It is broken out into grade levels, each student group for race/ethnicity, and even sex. But I do see there is a column called `total_indicator` that might be useful. I'd like to get a sense of what the values of that column are so I can determine if it could be used to filter the data somehow. To do that I will query the `distinct` values in that column.

In [19]:
pd.set_option('max_colwidth', 0) # this is to make sure pandas doesn't truncate the text in the column

query = """
select distinct
  total_indicator
from enrollment
"""

duckdb.sql(query).df()

,total_indicator
0,Category Set A - By Race/Ethnicity; Sex; Grade
1,Derived - Education Unit Total minus Adult Education Count
2,Derived - Subtotal by Race/Ethnicity and Sex minus Adult Education Count
3,Education Unit Total
4,Subtotal 4 - By Grade


We can see that there are two values that might work here:
- Education Unit Total
- Derived - Education Unit Total minus Adult Education Count

We can use the first as it seems more straight-forward but compare the enrollment number if we use the other one to better understand the differences.

Before we do that, let's pull out the relevant data from the dataset that we want to join to our directory data.

In [44]:
query = """
  select 
    school_year,
    statename,
    st,
    sch_name,
    st_schid,
    ncessch,
    student_count,
  from enrollment
  where total_indicator = 'Education Unit Total'
"""

total_enrollment = duckdb.sql(query).df()

In [45]:
total_enrollment

,school_year,statename,st,sch_name,st_schid,ncessch,student_count
0,2021-2022,ALABAMA,AL,Albertville Middle School,AL-101-0010,010000500870,920
1,2021-2022,ALABAMA,AL,Albertville High School,AL-101-0020,010000500871,1665
2,2021-2022,ALABAMA,AL,Albertville Intermediate School,AL-101-0110,010000500879,924
3,2021-2022,ALABAMA,AL,Albertville Elementary School,AL-101-0200,010000500889,891
4,2021-2022,ALABAMA,AL,Albertville Kindergarten and PreK,AL-101-0035,010000501616,579
...,...,...,...,...,...,...,...
100024,2021-2022,U.S. VIRGIN ISLANDS,VI,LOCKHART ELEMENTARY SCHOOL,VI-001-15,780003000024,284
100025,2021-2022,U.S. VIRGIN ISLANDS,VI,ULLA F MULLER ELEMENTARY SCHOOL,VI-001-17,780003000026,437
100026,2021-2022,U.S. VIRGIN ISLANDS,VI,YVONNE BOWSKY ELEMENTARY SCHOOL,VI-001-23,780003000027,381
100027,2021-2022,U.S. VIRGIN ISLANDS,VI,CANCRYN JUNIOR HIGH SCHOOL,VI-001-25,780003000033,744


I'm going to include a few different fields that may be necessary for joining such as the `school_year`, `statename`, and a couple of different identifiers since we're not 100% sure which one is unique enough to join yet.

We can do the same to the directory data and just keep the relevant fields for our analysis. 

In [66]:
query = """
  select
    school_year,
    statename,
    st,
    sch_name,
    st_schid,
    ncessch,
    charter_text
  from directory
"""
directory_info = duckdb.sql(query).df()

In [67]:
directory_info

,school_year,statename,st,sch_name,st_schid,ncessch,charter_text
0,2021-2022,ALABAMA,AL,Albertville Middle School,AL-101-0010,010000500870,No
1,2021-2022,ALABAMA,AL,Albertville High School,AL-101-0020,010000500871,No
2,2021-2022,ALABAMA,AL,Albertville Intermediate School,AL-101-0110,010000500879,No
3,2021-2022,ALABAMA,AL,Albertville Elementary School,AL-101-0200,010000500889,No
4,2021-2022,ALABAMA,AL,Albertville Kindergarten and PreK,AL-101-0035,010000501616,No
...,...,...,...,...,...,...,...
102125,2021-2022,U.S. VIRGIN ISLANDS,VI,LOCKHART ELEMENTARY SCHOOL,VI-001-15,780003000024,Not applicable
102126,2021-2022,U.S. VIRGIN ISLANDS,VI,ULLA F MULLER ELEMENTARY SCHOOL,VI-001-17,780003000026,Not applicable
102127,2021-2022,U.S. VIRGIN ISLANDS,VI,YVONNE BOWSKY ELEMENTARY SCHOOL,VI-001-23,780003000027,Not applicable
102128,2021-2022,U.S. VIRGIN ISLANDS,VI,CANCRYN JUNIOR HIGH SCHOOL,VI-001-25,780003000033,Not applicable


One thing to note is that the directory dataset has 102,130 rows and the enrollment dataset has 100,029 rows. So there is a difference of 2,101 rows between them. When we join them we should expect our returned data to be within those ranges if we join accurately and depending on the type of join used. 

For more info on the types of joins check out [this great tutorial from dataschool](https://dataschool.com/how-to-teach-people-sql/sql-join-types-explained-visually/).

In [84]:
query = """
  select
    e.school_year,
    e.statename,
    e.st,
    e.sch_name,
    e.st_schid,
    e.ncessch,
    cast(e.student_count as int) as student_count,
    d.charter_text
  from directory_info d
  inner join total_enrollment e
    on d.school_year = e.school_year
    and d.st = e.st
    and d.ncessch = e.ncessch
"""
enrollments_with_info = duckdb.sql(query).df()

In [85]:
enrollments_with_info

,school_year,statename,st,sch_name,st_schid,ncessch,student_count,charter_text
0,2021-2022,ALABAMA,AL,Albertville High School,AL-101-0020,010000500871,1665.0,No
1,2021-2022,ALABAMA,AL,Albertville Intermediate School,AL-101-0110,010000500879,924.0,No
2,2021-2022,ALABAMA,AL,Albertville Elementary School,AL-101-0200,010000500889,891.0,No
3,2021-2022,ALABAMA,AL,Albertville Kindergarten and PreK,AL-101-0035,010000501616,579.0,No
4,2021-2022,ALABAMA,AL,Albertville Primary School,AL-101-0005,010000502150,977.0,No
...,...,...,...,...,...,...,...,...
100024,2021-2022,PUERTO RICO,PR,ALEJANDRO JR CRUZ (GALATEO PARCELAS),PR-01-71449,720003002078,129.0,No
100025,2021-2022,PUERTO RICO,PR,BERWIND SUPERIOR,PR-01-67942,720003001333,228.0,No
100026,2021-2022,PUERTO RICO,PR,DR. ANTONIO S. PEDREIRA (ESPECIALIZADA),PR-01-61333,720003001370,299.0,No
100027,2021-2022,PUERTO RICO,PR,TRINA PADILLA DE SANZ,PR-01-61440,720003001379,425.0,No


The `inner join` is something like the intersection of a venn diagram. It will return records (based on a matching condition or set of conditions) that appear in both datasets. Whereas a `left join` would return all the records in the first dataset along with `null` values for any records that don't appear in the set that is being joined. This can be very useful when trying to determine which records are missing. For our purposes, we only only care about schools with actual enrollment (since we can't calculate non-existing numbers of students). 

```sql
  from directory_info d
  inner join total_enrollment e
    on d.school_year = e.school_year
    and d.st = e.st
    and d.ncessch = e.ncessch
```

Notice the use of table table aliases (`d` and `e` resprectively) to help distinsguish between columns in either dataset. Another way to write that join without the aliases would be:

```sql
  from directory_info
  inner join total_enrollment
    on directory_info.school_year = total_enrollment.school_year
    and directory_info.st = total_enrollment.st
    and directory_info.ncessch = total_enrollment.ncessch
```

For smaller table names this can be fine, but it does get tedious over time and sensible aliases can make the syntax of queries easier to read. 

Table aliases and explicit table references are technically only required when the columns share names across both datasets (since the SQL interpreter can't distinguish which one you mean) but it is generally considered a best practice when using joins to always specify the table source (preferably with an alias) in both the join conditions as well as in the `select` statement. 

Because both of our original dataset are constrained by year, we could probably remove some of the join conditions.

```sql
  from directory_info d
  inner join total_enrollment e
    on d.st = e.st
    and d.ncessch = e.ncessch
```

And if we are certain the NCES IDs are unique across states (they are), we could further simplify as:

```sql
  from directory_info d
  inner join total_enrollment e
    on d.ncessch = e.ncessch
```

## Calculating % of students in charters

Now that we've combined our datasets, we can write our aggregations similar to what we did for schools earlier. We could do this using CTEs or since we've stored the joined data in an output variable (as a dataframe) we can query that directly.

In [89]:
query = """
  select
    school_year,
    statename,
    count(if(charter_text = 'Yes', sch_name, null)) as charter_schools,
    count(sch_name) as total_schools,
    round((count(if(charter_text = 'Yes', sch_name, null))/count(sch_name)) * 100, 1) as percent_charter_schools,
    cast(sum(if(charter_text = 'Yes', student_count, 0)) as int) as charter_students,
    cast(sum(student_count) as int) as total_students,
    round((sum(if(charter_text = 'Yes', student_count, 0))/sum(student_count)) * 100, 1) as percent_charter_students,
  from enrollments_with_info 
  group by 
    school_year,
    statename
  having sum(student_count) is not null
    and sum(if(charter_text = 'Yes', student_count, 0)) > 0
  order by percent_charter_students desc
"""
charter_enrollments = duckdb.sql(query).df()

In [90]:
charter_enrollments

,school_year,statename,charter_schools,total_schools,percent_charter_schools,charter_students,total_students,percent_charter_students
0,2021-2022,DISTRICT OF COLUMBIA,123,240,51.2,39476,88543,44.6
1,2021-2022,ARIZONA,581,2418,24.0,231195,1131888,20.4
2,2021-2022,COLORADO,265,1941,13.7,130279,880582,14.8
3,2021-2022,NEVADA,93,746,12.5,63944,488251,13.1
4,2021-2022,FLORIDA,707,4191,16.9,361634,2832516,12.8
5,2021-2022,LOUISIANA,146,1366,10.7,86301,680793,12.7
6,2021-2022,DELAWARE,23,229,10.0,17201,139935,12.3
7,2021-2022,CALIFORNIA,1291,10167,12.7,678056,5874619,11.5
8,2021-2022,UTAH,137,1105,12.4,77733,687107,11.3
9,2021-2022,MICHIGAN,374,3538,10.6,150327,1396598,10.8


You'll notice some similar syntax as our previous aggregate query, but I have ommited the step convertin the percent to a string here to aid in future analysis.

## Wrapping up with some rankings

Now that we have the data we need for our investigation we can sort and re-sort the data and find answers to our inquiries, however one fundamental issue is coming up: the answer to our question is _it sort of depends ..._

Which is very common in data analysis. Depending on how you phrase your question you might get different answers. Wouldn't it be nice to have a definitive answer about which state, broadly, ranks highest for charter presence? 

To do this, let's explore a little more advanced of a topic in SQL queries: **window functions**.

I won't delve into them too deeply, but a basic understanding is helpful here:

> Imagine you're looking at a row of data in a table, like a line of people. A window function in SQL helps you calculate things for each person in the line, like finding their position compared to others, without changing the order. It's like having a magic window that lets you peek at everyone's info while keeping them in the same order they were in. This helps you figure out things like who's first, who's last, and who's in the middle without messing up the original line.

In this case, we'd like to understand the ranking of each state for each calculation we created earlier. I.E. who is the #1 state for most charter schools and who is the #1 state for the most students in charter schools, etc. For thank we will use the `rank()` window function. 

All window functions require an accompanying `over()` clause. They can get complicated but for our purposes we are just using them to specify the order to rank. 

In [94]:
query = """
  select *,
    rank() over(order by percent_charter_schools desc) as percent_schools_rank,
    rank() over(order by charter_schools desc) as count_schools_rank,
    rank() over(order by percent_charter_students desc) as percent_students_rank,
    rank() over(order by charter_students desc) as count_students_rank,
  from charter_enrollments
  order by statename
"""

rankings = duckdb.sql(query).df()

In [95]:
rankings

,school_year,statename,charter_schools,total_schools,percent_charter_schools,charter_students,total_students,percent_charter_students,percent_schools_rank,count_schools_rank,percent_students_rank,count_students_rank
0,2021-2022,ALABAMA,11,1494,0.7,3566,747846,0.5,41,39,43,38
1,2021-2022,ALASKA,30,500,6.0,7621,129944,5.9,29,34,27,35
2,2021-2022,ARIZONA,581,2418,24.0,231195,1131888,20.4,2,4,2,4
3,2021-2022,ARKANSAS,93,1084,8.6,42341,489565,8.6,20,22,14,25
4,2021-2022,CALIFORNIA,1291,10167,12.7,678056,5874619,11.5,5,1,8,1
5,2021-2022,COLORADO,265,1941,13.7,130279,880582,14.8,4,9,3,9
6,2021-2022,CONNECTICUT,21,1000,2.1,11047,496795,2.2,38,36,37,34
7,2021-2022,DELAWARE,23,229,10.0,17201,139935,12.3,17,35,7,31
8,2021-2022,DISTRICT OF COLUMBIA,123,240,51.2,39476,88543,44.6,1,17,1,26
9,2021-2022,FLORIDA,707,4191,16.9,361634,2832516,12.8,3,3,5,3


### Combining Rankings

These rankings are useful and can certainly make it easier to sort, but the haven't provided us with any more definitive of an answer. To do that, let's combine the rankings across metrics and find which state has the lowest value (ie. closest to #1 across all combined metrics).

We'll once again use the rank function but across the sum of the other ranks. 

In [99]:
query = """
  select *,
    rank() over(order by (percent_schools_rank + count_schools_rank + percent_students_rank + count_students_rank)) as combined_ranking
  from rankings
  order by combined_ranking
"""
duckdb.sql(query).df()

,school_year,statename,charter_schools,total_schools,percent_charter_schools,charter_students,total_students,percent_charter_students,percent_schools_rank,count_schools_rank,percent_students_rank,count_students_rank,combined_ranking
0,2021-2022,ARIZONA,581,2418,24.0,231195,1131888,20.4,2,4,2,4,1
1,2021-2022,FLORIDA,707,4191,16.9,361634,2832516,12.8,3,3,5,3,2
2,2021-2022,CALIFORNIA,1291,10167,12.7,678056,5874619,11.5,5,1,8,1,3
3,2021-2022,COLORADO,265,1941,13.7,130279,880582,14.8,4,9,3,9,4
4,2021-2022,TEXAS,1010,9105,11.1,443548,5428609,8.2,10,2,18,2,5
5,2021-2022,MICHIGAN,374,3538,10.6,150327,1396598,10.8,13,5,10,7,6
6,2021-2022,LOUISIANA,146,1366,10.7,86301,680793,12.7,12,13,6,11,7
7,2021-2022,UTAH,137,1105,12.4,77733,687107,11.3,9,14,9,12,8
8,2021-2022,DISTRICT OF COLUMBIA,123,240,51.2,39476,88543,44.6,1,17,1,26,9
9,2021-2022,NEVADA,93,746,12.5,63944,488251,13.1,8,22,4,15,10


## Conclusion

We can see that Arizona ranks highest for charter presence, followed by Florida. A quite different story than we were getting previously with our one-off questions. Fascinating!

There is much more you can do as you expand your knowledge of sql; including taking this data and visualizing it for presentation to an audience. That's a topic for another day but let's store our results in a new dataset.

In [106]:
query = """
  select *,
    rank() over(order by (percent_schools_rank + count_schools_rank + percent_students_rank + count_students_rank)) as combined_ranking
  from rankings
  order by combined_ranking
"""
charter_presence_rankings = duckdb.sql(query).df()
charter_presence_rankings.to_csv("charter_presence_rankings.csv.gz", compression="gzip", index=False)

You can [download this data here](/posts/sql-basics-with-duckdb/charter_presence_rankings.csv.gz) if you like. 

### Summary

In this tutorial we learned:
1. some basic SQL vocabulary
2. how to read data from files into dataframes for querying
3. how to explore the structure of a new dataset
4. how to select data from a dataframe using duckdb and sql syntax
5. how to filter and limit data
6. how to translate questions into queries
7. how to do some basic aggregations like `count` and `sum`
8. how to better organize complex queries with CTEs
9. a valuable lesson about data types
10. how to join data from multiple datasets
11. how to leverage a basic window function `rank` to come to a more definitive answer
12. how to store the output of our analysis for future use

I hope this was helpful and gave you a taste of how to iterate on a set of questions using SQL to find an answer in structured way (even without access to a database). 
